In [32]:
# Model Training
from datasets import load_from_disk

datasets = load_from_disk("./encoded_data4")
datasets

DatasetDict({
    train: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security on the Internet', '__index_level_0__'],
        num_rows: 644
    })
    valid: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', '

In [33]:
datasets = datasets.remove_columns(['__index_level_0__'])

In [34]:
datasets

DatasetDict({
    train: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security on the Internet'],
        num_rows: 644
    })
    valid: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security 

In [35]:
model_labels = [label for label in datasets['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [36]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [37]:
datasets["train"].column_names

['String',
 'Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [38]:
datasets["train"].features

{'String': Value(dtype='string', id=None),
 'Algorithm Design': Value(dtype='int32', id=None),
 'Basic Machine Organisation': Value(dtype='int32', id=None),
 'Computer System': Value(dtype='int32', id=None),
 'Data Manipulation and Analysis': Value(dtype='int32', id=None),
 'Data Organisation and Data Control': Value(dtype='int32', id=None),
 'Elementary Web Authoring': Value(dtype='int32', id=None),
 'Health and Ethical Issues': Value(dtype='int32', id=None),
 'Information Processing': Value(dtype='int32', id=None),
 'Intellectual Property': Value(dtype='int32', id=None),
 'Internet Services and Applications': Value(dtype='int32', id=None),
 'Multimedia Elements': Value(dtype='int32', id=None),
 'Networking and Internet Basics': Value(dtype='int32', id=None),
 'Program Development': Value(dtype='int32', id=None),
 'Spreadsheets and Databases': Value(dtype='int32', id=None),
 'Threats and Security on the Internet': Value(dtype='int32', id=None)}

In [39]:
#print(f"{datasets.train.column_names}\n{datasets.train.features}")

In [40]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [41]:
id2label = {idx:label for idx, label in enumerate(model_labels)}
label2id = {label:idx for idx, label in enumerate(model_labels)}

In [42]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np

tokenizer = RobertaTokenizer.from_pretrained("Zamachi/RoBERTa-for-multilabel-sentence-classification")

loading file vocab.json from cache at C:\Users\user/.cache\huggingface\hub\models--Zamachi--RoBERTa-for-multilabel-sentence-classification\snapshots\f8d542318fae3affd54b48577eab964e700d3f72\vocab.json
loading file merges.txt from cache at C:\Users\user/.cache\huggingface\hub\models--Zamachi--RoBERTa-for-multilabel-sentence-classification\snapshots\f8d542318fae3affd54b48577eab964e700d3f72\merges.txt
loading file added_tokens.json from cache at C:\Users\user/.cache\huggingface\hub\models--Zamachi--RoBERTa-for-multilabel-sentence-classification\snapshots\f8d542318fae3affd54b48577eab964e700d3f72\added_tokens.json
loading file special_tokens_map.json from cache at C:\Users\user/.cache\huggingface\hub\models--Zamachi--RoBERTa-for-multilabel-sentence-classification\snapshots\f8d542318fae3affd54b48577eab964e700d3f72\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\user/.cache\huggingface\hub\models--Zamachi--RoBERTa-for-multilabel-sentence-classification\snapsh

In [43]:
def encode_data(dataset):
    text = dataset["String"]
    # tokenize string
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=70)
    # create encoded array list with labels
    labelsBatch = {x: dataset[x] for x in dataset.keys() if x in model_labels}
    labels_matrix = np.zeros((len(text), len(model_labels)))
    for idx, label in enumerate(model_labels):
        labels_matrix[:, idx] = labelsBatch[label]
    
    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [44]:
# tokenize the dataset
encoded_ds = datasets.map(encode_data, batched=True, remove_columns=datasets['train'].column_names)

Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data4\train\cache-9cdeced0d371d054.arrow
100%|██████████| 1/1 [00:00<00:00, 41.78ba/s]


In [45]:
encoded_ds['train']['labels'][0]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

In [46]:
[id2label[idx] for idx, label in enumerate(encoded_ds['train'][15]['labels']) if label == 1.0]

['Data Organisation and Data Control']

In [47]:
import torch
from torch import cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

encoded_ds.set_format("torch")

Using device: cuda



In [48]:
encoded_ds["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 644
})

In [49]:

# define model
model = RobertaForSequenceClassification.from_pretrained("Zamachi/RoBERTa-for-multilabel-sentence-classification", problem_type="multi_label_classification", num_labels=len(model_labels), id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

loading configuration file config.json from cache at C:\Users\user/.cache\huggingface\hub\models--Zamachi--RoBERTa-for-multilabel-sentence-classification\snapshots\f8d542318fae3affd54b48577eab964e700d3f72\config.json
Model config RobertaConfig {
  "_name_or_path": "c:\\Users\\Zamachi\\Projects\\Finals\\RoBERTa-for-multilabel-sentence-classification",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Algorithm Design",
    "1": "Basic Machine Organisation",
    "2": "Computer System",
    "3": "Data Manipulation and Analysis",
    "4": "Data Organisation and Data Control",
    "5": "Elementary Web Authoring",
    "6": "Health and Ethical Issues",
    "7": "Information Processing",
    "8": "Intellectual Property",
    "9": "Internet Services and Applications",
  

In [50]:
# train model
batch_size = 8
metric_name = "f1"

In [51]:
import evaluate
metric  = evaluate.load("accuracy")

In [52]:
import time

train_name_dir = time.strftime(f"%Y%m%d_%H%M%S$-train-model-{model.name_or_path}")
train_name_dir

'20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification'

In [53]:
from transformers import TrainingArguments, Trainer


args = TrainingArguments(
    output_dir="./RBert-v1/"+ train_name_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 81,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='./RBert-v1/logging/'+ train_name_dir
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [54]:
args.learning_rate

2e-05

In [55]:
# from transformers import AdamW
# optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

In [56]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, balanced_accuracy_score, hamming_loss
from transformers import EvalPrediction


def multi_label_metrics(predictions, labels, threshold=0.50):
    # apply sigmoid on predictions fitting (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    # logits, labels = predictions
    # prediction = np.argmax(logits, axis=-1)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'balanced_accuracy': balanced_accuracy,
               }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

def compute_metrics_new(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [57]:
encoded_ds['train'][0]['labels'].type()

'torch.FloatTensor'

In [58]:
# model.to(device)

In [59]:
outputs = model(input_ids=encoded_ds['train']['input_ids'][0].unsqueeze(0), labels=encoded_ds['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7616, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.4198, -0.0892,  0.0682, -0.0094,  0.1213,  0.1981, -0.1380, -0.2670,
          0.4694,  0.2932, -0.2617, -0.0059,  0.2165,  0.0412,  0.2677]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [60]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [61]:
trainer.train()

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 644
  Num Epochs = 81
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6561
  Number of trainable parameters = 124670991
***** Running Evaluation *****
  Num examples = 81
  Batch size = 8


                                                   

                                        
 98%|█████████▊| 6399/6561 [47:26<00:14, 11.20it/s]

Saving model checkpoint to ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-81
Configuration sav

{'eval_loss': 0.2531745731830597, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_balanced_accuracy': 0.06666666666666667, 'eval_runtime': 0.2433, 'eval_samples_per_second': 332.944, 'eval_steps_per_second': 45.215, 'epoch': 1.0}


Model weights saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-81\pytorch_model.bin
tokenizer config file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-81\tokenizer_config.json
Special tokens file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-81\special_tokens_map.json
added tokens file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-81\added_tokens.json
***** Running Evaluation *****
  Num examples = 81
  Batch size = 8


                                                   
                                               

 98%|█████████▊| 6399/6561 [47:44<00:14, 11.20it/s]

Saving model checkpoint to ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-162
Configurati

{'eval_loss': 0.2535138726234436, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_balanced_accuracy': 0.06666666666666667, 'eval_runtime': 0.2221, 'eval_samples_per_second': 364.737, 'eval_steps_per_second': 49.532, 'epoch': 2.0}


Model weights saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-162\pytorch_model.bin
tokenizer config file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-162\tokenizer_config.json
Special tokens file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-162\special_tokens_map.json
added tokens file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-162\added_tokens.json
***** Running Evaluation *****
  Num examples = 81
  Batch size = 8


                                                   
                                               

 98%|█████████▊| 6399/6561 [48:00<00:14, 11.20it/s]

Saving model checkpoint to ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-243
Configu

...

Model weights saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-324\pytorch_model.bin
tokenizer config file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-324\tokenizer_config.json
Special tokens file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-324\special_tokens_map.json
added tokens file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-324\added_tokens.json
***** Running Evaluation *****
  Num examples = 81
  Batch size = 8


                                                   
                                               

 98%|█████████▊| 6399/6561 [48:34<00:14, 11.20it/s]

Saving model checkpoint to ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-405
Configu

{'eval_loss': 0.2560178339481354, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_balanced_accuracy': 0.06666666666666667, 'eval_runtime': 0.227, 'eval_samples_per_second': 356.887, 'eval_steps_per_second': 48.466, 'epoch': 5.0}


Model weights saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-405\pytorch_model.bin
tokenizer config file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-405\tokenizer_config.json
Special tokens file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-405\special_tokens_map.json
added tokens file saved in ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-405\added_tokens.json
***** Running Evaluation *****
  Num examples = 81
  Batch size = 8


                                                   
                                               

 98%|█████████▊| 6399/6561 [48:58<00:14, 11.20it/s]

Saving model checkpoint to ./RBert-v1/20230329_213955$-train-model-Zamachi/RoBERTa-for-multilabel-sentence-classification\checkpoint-486
Configu

{'eval_loss': 0.26556873321533203, 'eval_f1': 0.17699115044247787, 'eval_roc_auc': 0.5506107483469628, 'eval_accuracy': 0.09876543209876543, 'eval_balanced_accuracy': 0.08888888888888888, 'eval_runtime': 0.2289, 'eval_samples_per_second': 353.812, 'eval_steps_per_second': 48.049, 'epoch': 6.0}


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
prediction_result = trainer.predict(encoded_ds['test'])

In [ ]:
print(prediction_result.metrics)

In [ ]:
trainer.save_model("./RBert-model-v1")

In [ ]:
tokenizer.save_pretrained("./RBert-model-v1/tokenizer")